In [461]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Merge the databases that have information about RNA interactions

## Imports

In [462]:
import os
import pandas as pd


In [463]:
fn_RNAInter = os.path.join('..', 'data', 'sRNA', 'RNAInter', 'Download_data_RR.csv')
fn_srnatarbase = os.path.join('..', 'data', 'sRNA', 'sRNATarBase', 'sRNATarBase.csv')
fn_merged = os.path.join('..', 'data', 'sRNA', 'merged_EcoCyc_RNAInter_sRNATarBase.csv')
data_rnainter = pd.read_csv(fn_RNAInter)
data_srnatarbase = pd.read_csv(fn_srnatarbase)
data_merged = pd.read_csv(fn_merged)
try:
    data_rnainter = data_rnainter.drop(columns=['Unnamed: 0', 'level_0'])
    data_srnatarbase = data_srnatarbase.drop(columns=['Unnamed: 0'])
    data_merged = data_merged.drop(columns=['Unnamed: 0'])
except:
    pass


In [464]:
print('Overlapping:', len(set([s for s in data_srnatarbase['sRNA'] if (s in data_rnainter[data_rnainter['Category1'] == 'sRNA']
      ['Interactor1.Symbol'].unique()) or (s in data_rnainter[data_rnainter['Category2'] == 'sRNA']['Interactor2.Symbol'].unique())])))
print('Non-overlapping:', len(set([s for s in data_srnatarbase['sRNA'] if (
    s not in data_rnainter[data_rnainter['Category1'] == 'sRNA']['Interactor1.Symbol'].unique()) and (s not in data_rnainter[data_rnainter['Category1'] == 'sRNA']['Interactor2.Symbol'].unique())])))

print('Overlapping:', len(set([s for s in data_rnainter[data_rnainter['Category1'] == 'sRNA']['Interactor1.Symbol'] if (
    s in data_srnatarbase['sRNA'].unique())])) +
    len(set([s for s in data_rnainter[data_rnainter['Category2'] == 'sRNA']['Interactor2.Symbol'] if (
        s not in data_srnatarbase['sRNA'].unique())])))

Overlapping: 36
Non-overlapping: 32
Overlapping: 23


In [465]:
data_rnainter.head()

,index,RNAInterID,Interactor1.Symbol,Category1,Species1,Interactor2.Symbol,Category2,Species2,Raw_ID1,Raw_ID2,score,strong,weak,predict,Sequence1,Sequence2
0,1029724,RR05384747,acnA,mRNA,Escherichia coli str. K-12 substr. MG1655,ryhB,sRNA,Escherichia coli str. K-12 substr. MG1655,NCBI:946724,NCBI:2847761,0.2292,Northern blot//Reporter assay,NaN,NaN,ATTCGGAACGAGGCCTGAAGCAGTGTCGCCGTCCCTCTGCCTTGCA...,TTTGAGAGCGTCTCTGTCCCTCGTTTTGCGGTTAAGCCGCATCCAT...
1,1029725,RR05384845,acrZ,mRNA,Escherichia coli str. K-12 substr. MG1655,omrB,sRNA,Escherichia coli str. K-12 substr. MG1655,NCBI:945365,NCBI:2847747,0.1778,Northern blot,Microarray,NaN,ACTTACTACTGTCTTCGGGGGGTCCGAGGTTTCTGGGGGGTCGTAC...,TGTTCTATACTTGGGTTCGACTTGGGTTAGACTTGTCTTTACTGTC...
2,1029732,RR05387056,arcZ,sRNA,Escherichia coli str. K-12 substr. MG1655,flhD,mRNA,Escherichia coli str. K-12 substr. MG1655,NCBI:2847690,NCBI:945442,0.2314,Northern blot//Reporter assay,NaN,NaN,CTCATGTTGACCGCTTGTTTAGCAGCTTCAAGGAAGCTGAAGGGCA...,CTCGTGCCGAATTCGGCACGAGCGATATTTCATCAGTTATCGGTAA...
3,1029733,RR05387057,arcZ,sRNA,Escherichia coli str. K-12 substr. MG1655,rpoS,mRNA,Escherichia coli str. K-12 substr. MG1655,NCBI:2847690,NCBI:947210,0.1996,Northern blot//RACE//RT-PCR//Beta-galactosidas...,NaN,NaN,CTCATGTTGACCGCTTGTTTAGCAGCTTCAAGGAAGCTGAAGGGCA...,ATCCTCGGGTCTTGCAGGCCACACAGGACACCCTGAACCGTCATGG...
4,1029734,RR05387126,argR,mRNA,Escherichia coli str. K-12 substr. MG1655,dsrA,sRNA,Escherichia coli str. K-12 substr. MG1655,NCBI:947861,NCBI:946470,0.2139,Primer extension assay,NaN,NaN,GACAATGGCGATAGTATGGCGGTTGTTTCTTTCCCATCTCTACTCA...,TGAGCAACTTTATTCACATAATTTCTACACCAAGAACTCGAGGTTA...


In [466]:
data_srnatarbase.head()

,Target,sRNA,Target ID,sRNA ID,Regulation,Target Type,sRNA Type,Target Binding Position,sRNA Binding Position,Target Sequence,sRNA Sequence
0,hns,dsrA,"chromosome:NC_000913.3, Gene ID:945829","chromosome:NC_000913.3, Gene ID:946470",Repression,mRNA,trans-encoded antisense RNA,"['7..19', 'NA', '7..19; 401..411']","['31..43', 'NA', '31..54']",ATGAGCGAAGCACTTAAAATTCTGAACAACATCCGTACTCTTCGTG...,AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...
1,rbsD,dsrA,"chromosome:NC_000913.3, Gene ID:948267","chromosome:NC_000913.3, Gene ID:946470",Repression,mRNA,trans-encoded antisense RNA,['NA'],['NA'],ATGAAAAAAGGCACCGTTCTTAATTCTGATATTTCATCGGTGATCT...,AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...
2,argR,dsrA,"chromosome:NC_000913.3, Gene ID:947861","chromosome:NC_000913.3, Gene ID:946470",Repression,mRNA,trans-encoded antisense RNA,['NA'],['NA'],ATGCGAAGCTCGGCTAAGCAAGAAGAACTAGTTAAAGCATTTAAAG...,AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...
3,ilvI,dsrA,"chromosome:NC_000913.3, Gene ID:948793","chromosome:NC_000913.3, Gene ID:946470",Repression,mRNA,trans-encoded antisense RNA,['NA'],['NA'],ATGGAGATGTTGTCTGGAGCCGAGATGGTCGTCCGATCGCTTATCG...,AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...
4,rpoS,dsrA,"chromosome:NC_000913.3, Gene ID:947210","chromosome:NC_000913.3, Gene ID:946470",Induction,mRNA,trans-encoded antisense RNA,"['-119..-97', 'NA', 'NA']","['10..32', 'NA', 'NA']",ATGAGTCAGAATACGCTGAAAGTTCATGATTTAAATGAAGATGCGG...,AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...


In [467]:
data_srnatarbase[data_srnatarbase['Target Sequence'].isna() | data_srnatarbase['sRNA Sequence'].isna()]

,Target,sRNA,Target ID,sRNA ID,Regulation,Target Type,sRNA Type,Target Binding Position,sRNA Binding Position,Target Sequence,sRNA Sequence
161,traJ,FinP,"chromosome:NC_000913.3, Gene ID:","chromosome:NC_000913.3, Gene ID:",Repression,mRNA,trans-encoded antisense RNA,['NA'],['NA'],NaN,NaN
260,rpoS,IS118,"chromosome:NC_000913.3, Gene ID:947210","chromosome:NC_000913.3, Gene ID:",No Interaction,mRNA,NaN,['NA'],['NA'],ATGAGTCAGAATACGCTGAAAGTTCATGATTTAAATGAAGATGCGG...,NaN


# Rename columns

In [468]:
cols = ['Interactor 1', 'Interactor 2', 'ID 1', 'ID 2', 'Category 1', 'Category 2', 'Regulation', 'Binding position 1', 'Binding position 2', 'Sequence 1', 'Sequence 2', 'Source DB']
cols_srnatarbase = {
    'sRNA': 'Interactor 1',
    'Target': 'Interactor 2',
    'sRNA ID': 'ID 1',
    'Target ID': 'ID 2',
    'sRNA Type': 'Category 1',
    'Target Type': 'Category 2', 
    'sRNA Binding Position': 'Binding position 1',
    'Target Binding Position': 'Binding position 2',
    'sRNA Sequence': 'Sequence 1',
    'Target Sequence': 'Sequence 2'
}
cols_rnainter = {
    'Interactor1.Symbol': 'Interactor 1',
    'Interactor2.Symbol': 'Interactor 2',
    'Raw_ID1': 'ID 1',
    'Raw_ID2': 'ID 2',
    'Category1': 'Category 1',
    'Category2': 'Category 2', 
    'Sequence1': 'Sequence 1',
    'Sequence2': 'Sequence 2'
}
data_rnainter = data_rnainter.rename(columns=cols_rnainter)
data_srnatarbase = data_srnatarbase.rename(columns=cols_srnatarbase)

data_rnainter['Regulation'] = ''
data_rnainter['Binding position 1'] = ''
data_rnainter['Binding position 2'] = ''
data_rnainter['Source DB'] = 'RNAInter'

data_srnatarbase['Source DB'] = 'sRNATarBase'

In [469]:
data = pd.concat([data_rnainter[cols], data_srnatarbase[cols]], axis=0)
data = data.reset_index(drop=True)
data

,Interactor 1,Interactor 2,ID 1,ID 2,Category 1,Category 2,Regulation,Binding position 1,Binding position 2,Sequence 1,Sequence 2,Source DB
0,acnA,ryhB,NCBI:946724,NCBI:2847761,mRNA,sRNA,,,,ATTCGGAACGAGGCCTGAAGCAGTGTCGCCGTCCCTCTGCCTTGCA...,TTTGAGAGCGTCTCTGTCCCTCGTTTTGCGGTTAAGCCGCATCCAT...,RNAInter
1,acrZ,omrB,NCBI:945365,NCBI:2847747,mRNA,sRNA,,,,ACTTACTACTGTCTTCGGGGGGTCCGAGGTTTCTGGGGGGTCGTAC...,TGTTCTATACTTGGGTTCGACTTGGGTTAGACTTGTCTTTACTGTC...,RNAInter
2,arcZ,flhD,NCBI:2847690,NCBI:945442,sRNA,mRNA,,,,CTCATGTTGACCGCTTGTTTAGCAGCTTCAAGGAAGCTGAAGGGCA...,CTCGTGCCGAATTCGGCACGAGCGATATTTCATCAGTTATCGGTAA...,RNAInter
3,arcZ,rpoS,NCBI:2847690,NCBI:947210,sRNA,mRNA,,,,CTCATGTTGACCGCTTGTTTAGCAGCTTCAAGGAAGCTGAAGGGCA...,ATCCTCGGGTCTTGCAGGCCACACAGGACACCCTGAACCGTCATGG...,RNAInter
4,argR,dsrA,NCBI:947861,NCBI:946470,mRNA,sRNA,,,,GACAATGGCGATAGTATGGCGGTTGTTTCTTTCCCATCTCTACTCA...,TGAGCAACTTTATTCACATAATTTCTACACCAAGAACTCGAGGTTA...,RNAInter
...,...,...,...,...,...,...,...,...,...,...,...,...
760,ryhB,msrA,"chromosome:NC_000913.3, Gene ID:2847761","chromosome:NC_000913.3, Gene ID:948734",NaN,mRNA,No Interaction,['NA'],['NA'],GCGATCAGGAAGACCCTCGCGGAGAACCTGAAAGCACGACATTGCT...,ATGAGTTTATTTGATAAAAAGCATCTGGTTTCCCCCGCCGATGCCC...,sRNATarBase
761,gcvB,hdeA,"chromosome:NC_000913.3, Gene ID:2847720","chromosome:NC_000913.3, Gene ID:948025",trans-encoded antisense RNA,mRNA,Induction,['NA'],['NA'],ACTTCCTGAGCCGGAACGAAAAGTTTTATCGGAATGCGTGTTCTGG...,ATGAAAAAAGTATTAGGCGTTATTCTTGGTGGTCTGCTTCTTCTGC...,sRNATarBase
762,gcvB,hdeB,"chromosome:NC_000913.3, Gene ID:2847720","chromosome:NC_000913.3, Gene ID:948026",trans-encoded antisense RNA,mRNA,Induction,['NA'],['NA'],ACTTCCTGAGCCGGAACGAAAAGTTTTATCGGAATGCGTGTTCTGG...,ATGAATATTTCATCTCTCCGTAAAGCGTTTATTTTTATGGGCGCTG...,sRNATarBase
763,Esr41,fliC,"chromosome:NC_002695.1, Gene ID:","chromosome:NC_002695.1, Gene ID:",trans-encoded antisense RNA,mRNA,Induction,['NA'],['NA'],GATGCTCTAGGCATCACATTTTCTCCATGGGGTATTCCCTCCGCCG...,ATGGCACAAGTCATTAATACCAACAGCCTCTCGCTGATCACTCAAA...,sRNATarBase


In [470]:
data[(data['Interactor 1'] == 'chiX') | (data['Interactor 2'] == 'chiX')]


,Interactor 1,Interactor 2,ID 1,ID 2,Category 1,Category 2,Regulation,Binding position 1,Binding position 2,Sequence 1,Sequence 2,Source DB
9,chbC,chiX,NCBI:945982,NCBI:5061500,mRNA,sRNA,,,,NCTATCATACTTTAGAAAAGCCATAGAGGGAAATTTTGTGAATAAG...,TGGATTGAACCNTCTTACACCCTGCTACACGCATCCTTAAGTCACC...,RNAInter
11,chiP,chiX,NCBI:945296,NCBI:5061500,mRNA,sRNA,,,,GCCTGTGACGGGGTCGGTTTGTACACCGATAAACTCAT,TGGATTGAACCNTCTTACACCCTGCTACACGCATCCTTAAGTCACC...,RNAInter
12,chiX,citA,NCBI:5061500,NCBI:945233,sRNA,mRNA,,,,TGGATTGAACCNTCTTACACCCTGCTACACGCATCCTTAAGTCACC...,GCCTGTGACCGACGTACTGGGGGATACGCAGATAAACTCAT,RNAInter
13,chiX,rpoS,NCBI:5061500,NCBI:947210,sRNA,mRNA,,,,TGGATTGAACCNTCTTACACCCTGCTACACGCATCCTTAAGTCACC...,ATCCTCGGGTCTTGCAGGCCACACAGGACACCCTGAACCGTCATGG...,RNAInter
595,chiX,rpoS,"chromosome:NC_000913.3, Gene ID:5061500","chromosome:NC_000913.3, Gene ID:947210",NaN,mRNA,No Interaction,['NA'],['NA'],ACACCGTCGCTTAAAGTGACGGCATAATAATAAAAAAATGAAATTC...,ATGAGTCAGAATACGCTGAAAGTTCATGATTTAAATGAAGATGCGG...,sRNATarBase
669,chiX,citA,"chromosome:NC_000913.3, Gene ID:5061500","chromosome:NC_000913.3, Gene ID:945233",trans-encoded antisense RNA,mRNA,Repression,['46..57'],['-37..-26'],ACACCGTCGCTTAAAGTGACGGCATAATAATAAAAAAATGAAATTC...,ATGTTGCAGCTTAACGAGAATAAACAGTTTGCATTTTTCCAAAGAC...,sRNATarBase
690,chiX,chiP,"chromosome:NC_000913.3, Gene ID:5061500","chromosome:NC_000913.3, Gene ID:945296",trans-encoded antisense RNA,mRNA,Repression,['81..92'],['45..56'],ACACCGTCGCTTAAAGTGACGGCATAATAATAAAAAAATGAAATTC...,ATGCGTACGTTTAGTGGCAAACGTAGTACGCTGGCGCTGGCTATCG...,sRNATarBase
724,chiX,chbC,"chromosome:NC_000913.3, Gene ID:5061500","chromosome:NC_000913.3, Gene ID:945982",trans-encoded antisense RNA,mRNA,Repression,['NA'],['NA'],ACACCGTCGCTTAAAGTGACGGCATAATAATAAAAAAATGAAATTC...,ATGAGTAATGTTATTGCATCGCTTGAAAAGGTACTCCTCCCTTTTG...,sRNATarBase


In [471]:
data[data['Sequence 1'].isna() | data['Sequence 2'].isna()]

,Interactor 1,Interactor 2,ID 1,ID 2,Category 1,Category 2,Regulation,Binding position 1,Binding position 2,Sequence 1,Sequence 2,Source DB
181,mdtF,oxyS,NCBI:948030,NCBI:2847701,mRNA,sRNA,,,,NaN,TTTGCGAACTTATCGGTTTCGTATTGAGGGCAATAAACTCTCGAGG...,RNAInter
258,omrA,rrrD,NCBI:2847746,NCBI:947539,sRNA,mRNA,,,,CGCTCGAATCTGAGCCCCTACTCTCGTTACCATGTAACGGCGTACT...,NaN,RNAInter
515,FinP,traJ,"chromosome:NC_000913.3, Gene ID:","chromosome:NC_000913.3, Gene ID:",trans-encoded antisense RNA,mRNA,Repression,['NA'],['NA'],NaN,NaN,sRNATarBase
614,IS118,rpoS,"chromosome:NC_000913.3, Gene ID:","chromosome:NC_000913.3, Gene ID:947210",NaN,mRNA,No Interaction,['NA'],['NA'],NaN,ATGAGTCAGAATACGCTGAAAGTTCATGATTTAAATGAAGATGCGG...,sRNATarBase


## Sort interactions alphabetically to get rid of duplicates

In [472]:
cols = ['Interactor', 'ID', 'Category', 'Binding position', 'Sequence']

for interactors in data[['Interactor 1', 'Interactor 2']].iterrows():
    
    i1, i2 = interactors[1]
    
    cat1, cat2 = data.iloc[interactors[0]][['Category 1', 'Category 2']].to_list()
    sorted_interactors = [i1, i2] if cat2 == 'mRNA' else [i2, i1]

    if cat2 != 'mRNA':
        
        that_i = 1
        this_i = 2
        
        old = {}
        old[this_i] = data[[c + ' ' + str(this_i) for c in cols]].iloc[interactors[0]].to_list()
        old[that_i] = data[[c + ' ' + str(that_i) for c in cols]].iloc[interactors[0]].to_list()
        
        for this_i, that_i in [(1, 2), (2, 1)]:
            if (len(data[(data[f'Interactor {this_i}'] == sorted_interactors[that_i - 1]) & (
                        data[f'Interactor {that_i}'] == sorted_interactors[this_i - 1])][[
                  c + ' ' + str(that_i) for c in cols]])) > 1:
                print(i1, i2)
            data.loc[interactors[0], [c + ' ' + str(this_i) for c in cols]] = old[that_i]

omrA yzgL
omrB ygaY
ryhB yagJ


# Correct for RNAInter (false) sequences

In [473]:

for i, s in [('Interactor 1', 'Sequence 1'), ('Interactor 2', 'Sequence 2')]:
    for rna, seq in data[[i, s]].value_counts().index.tolist():
        
        new_seq = seq
        
        if (rna in data_srnatarbase['Interactor 1'].to_list()):
            if data_srnatarbase[data_srnatarbase['Interactor 1'] == rna]['Sequence 1'].iloc[0] != seq:
                new_seq = data_srnatarbase[data_srnatarbase['Interactor 1'] == rna]['Sequence 1'].iloc[0]
                
        elif (rna in data_srnatarbase['Interactor 2'].to_list()):
            if data_srnatarbase[data_srnatarbase['Interactor 2'] == rna]['Sequence 2'].iloc[0] != seq:
                new_seq = data_srnatarbase[data_srnatarbase['Interactor 2'] == rna]['Sequence 2'].iloc[0] 
            
        elif (rna in data_merged['Symbol'].to_list()):
            if (data_merged[data_merged['Symbol'] == rna]['Sequence'].iloc[0] != seq):
                new_seq = data_merged[data_merged['Symbol'] == rna]['Sequence'].iloc[0]
        else: 
            print('Could not correct the sequence for', rna)
            
        if type(new_seq) == str:
            data.loc[data[i] == rna, s] = new_seq
        
            

In [474]:
data[data['Sequence 1'].isna()]

,Interactor 1,Interactor 2,ID 1,ID 2,Category 1,Category 2,Regulation,Binding position 1,Binding position 2,Sequence 1,Sequence 2,Source DB
515,FinP,traJ,"chromosome:NC_000913.3, Gene ID:","chromosome:NC_000913.3, Gene ID:",trans-encoded antisense RNA,mRNA,Repression,['NA'],['NA'],NaN,NaN,sRNATarBase
614,IS118,rpoS,"chromosome:NC_000913.3, Gene ID:","chromosome:NC_000913.3, Gene ID:947210",NaN,mRNA,No Interaction,['NA'],['NA'],NaN,ATGAGTCAGAATACGCTGAAAGTTCATGATTTAAATGAAGATGCGG...,sRNATarBase


# Drop duplicates

In [475]:
sum(data[['Interactor 1', 'Interactor 2']].value_counts() > 1)

316

In [476]:
data[['Interactor 1', 'Interactor 2']].value_counts()

Interactor 1  Interactor 2
omrA          hisM            2
              clpB            2
              higB            2
              gntP            2
              gmhB            2
                             ..
micA          lpxT            1
              luxS            1
              ompA            1
              ompC            1
yzgL          omrA            1
Name: count, Length: 449, dtype: int64

In [477]:
for d in data[['Interactor 1', 'Interactor 2']].value_counts()[data[['Interactor 1', 'Interactor 2']].value_counts() > 1].index:
    
    data = data.drop(index=data[(data['Interactor 1'] == d[0]) & (data['Interactor 2'] == d[1]) & (data['Regulation'] == '')].index[0])
    
# data[data['Symbol'] == d].index[0]    
    

In [478]:
data[(data['Interactor 1'] == 'chiX') | (data['Interactor 2'] == 'chiX')]


,Interactor 1,Interactor 2,ID 1,ID 2,Category 1,Category 2,Regulation,Binding position 1,Binding position 2,Sequence 1,Sequence 2,Source DB
595,chiX,rpoS,"chromosome:NC_000913.3, Gene ID:5061500","chromosome:NC_000913.3, Gene ID:947210",NaN,mRNA,No Interaction,['NA'],['NA'],ACACCGTCGCTTAAAGTGACGGCATAATAATAAAAAAATGAAATTC...,ATGAGTCAGAATACGCTGAAAGTTCATGATTTAAATGAAGATGCGG...,sRNATarBase
669,chiX,citA,"chromosome:NC_000913.3, Gene ID:5061500","chromosome:NC_000913.3, Gene ID:945233",trans-encoded antisense RNA,mRNA,Repression,['46..57'],['-37..-26'],ACACCGTCGCTTAAAGTGACGGCATAATAATAAAAAAATGAAATTC...,ATGTTGCAGCTTAACGAGAATAAACAGTTTGCATTTTTCCAAAGAC...,sRNATarBase
690,chiX,chiP,"chromosome:NC_000913.3, Gene ID:5061500","chromosome:NC_000913.3, Gene ID:945296",trans-encoded antisense RNA,mRNA,Repression,['81..92'],['45..56'],ACACCGTCGCTTAAAGTGACGGCATAATAATAAAAAAATGAAATTC...,ATGCGTACGTTTAGTGGCAAACGTAGTACGCTGGCGCTGGCTATCG...,sRNATarBase
724,chiX,chbC,"chromosome:NC_000913.3, Gene ID:5061500","chromosome:NC_000913.3, Gene ID:945982",trans-encoded antisense RNA,mRNA,Repression,['NA'],['NA'],ACACCGTCGCTTAAAGTGACGGCATAATAATAAAAAAATGAAATTC...,ATGAGTAATGTTATTGCATCGCTTGAAAAGGTACTCCTCCCTTTTG...,sRNATarBase


In [479]:
data

,Interactor 1,Interactor 2,ID 1,ID 2,Category 1,Category 2,Regulation,Binding position 1,Binding position 2,Sequence 1,Sequence 2,Source DB
44,micA,dppA,NCBI:2847697,NCBI:948062,sRNA,mRNA,,,,CAAAAATGGTGTCGCTCGTATTATTGTCGAGAAACCTTTCGGCAAG...,ATGCGTATTTCCTTGAAAAAGTCAGGGATGCTGAAGCTTGGTCTCA...,RNAInter
50,dsrA,dsrB,NCBI:946470,NCBI:946468,sRNA,mRNA,,,,AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...,TAAATGNAAACAAGTTTTATTTTTATCATCAGAACATTCTAATAAA...,RNAInter
53,dsrA,ilvH,NCBI:946470,NCBI:947267,sRNA,mRNA,,,,AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...,ATTCGCCACAACAAGGACAGGAAGGTTCGGCGCAAGGAGCCCAAGA...,RNAInter
63,micA,ecnB,NCBI:2847697,NCBI:2847737,sRNA,mRNA,,,,CAAAAATGGTGTCGCTCGTATTATTGTCGAGAAACCTTTCGGCAAG...,ATGGTGAAGAAGACAATTGCAGCGATCTTTTCTGTTCTGGTGCTTT...,RNAInter
74,oxyS,fhlA,NCBI:2847701,NCBI:947181,sRNA,mRNA,,,,GAAACGGAGCGGCACCTCTTTTAACCCTTGAAGTCACTGCCCGTTT...,CACCAGATTGCCAAGCTGATGGTCGAGCTGTCCAAATCCCAGGATG...,RNAInter
...,...,...,...,...,...,...,...,...,...,...,...,...
760,ryhB,msrA,"chromosome:NC_000913.3, Gene ID:2847761","chromosome:NC_000913.3, Gene ID:948734",NaN,mRNA,No Interaction,['NA'],['NA'],GCGATCAGGAAGACCCTCGCGGAGAACCTGAAAGCACGACATTGCT...,ATGAGTTTATTTGATAAAAAGCATCTGGTTTCCCCCGCCGATGCCC...,sRNATarBase
761,gcvB,hdeA,"chromosome:NC_000913.3, Gene ID:2847720","chromosome:NC_000913.3, Gene ID:948025",trans-encoded antisense RNA,mRNA,Induction,['NA'],['NA'],ACTTCCTGAGCCGGAACGAAAAGTTTTATCGGAATGCGTGTTCTGG...,ATGAAAAAAGTATTAGGCGTTATTCTTGGTGGTCTGCTTCTTCTGC...,sRNATarBase
762,gcvB,hdeB,"chromosome:NC_000913.3, Gene ID:2847720","chromosome:NC_000913.3, Gene ID:948026",trans-encoded antisense RNA,mRNA,Induction,['NA'],['NA'],ACTTCCTGAGCCGGAACGAAAAGTTTTATCGGAATGCGTGTTCTGG...,ATGAATATTTCATCTCTCCGTAAAGCGTTTATTTTTATGGGCGCTG...,sRNATarBase
763,Esr41,fliC,"chromosome:NC_002695.1, Gene ID:","chromosome:NC_002695.1, Gene ID:",trans-encoded antisense RNA,mRNA,Induction,['NA'],['NA'],GATGCTCTAGGCATCACATTTTCTCCATGGGGTATTCCCTCCGCCG...,ATGGCACAAGTCATTAATACCAACAGCCTCTCGCTGATCACTCAAA...,sRNATarBase


In [480]:
data.to_csv(os.path.join('..', 'data', 'sRNA', 'merged_inter.csv'))